<a href="https://colab.research.google.com/github/AlvinaAnoop/Firstrepo/blob/main/halfsempro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install OpenCV
!pip install opencv-python-headless

# Install TensorFlow
!pip install tensorflow

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load the dataset from Google Drive
fer2013_url = '/content/drive/MyDrive/fer2013.csv'
data = pd.read_csv(fer2013_url)

# Convert pixel values to image arrays
data['pixels'] = data['pixels'].apply(lambda x: np.array(x.split()).astype('float32'))

# For FER-2013, the data is already clean

# Normalize pixel values
data['pixels'] /= 255.0

# Split the data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['pixels'], data['emotion'], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Convert pixel values to image arrays and reshape
X_train = np.array([np.reshape(x, (48, 48, 1)) for x in X_train])
X_val = np.array([np.reshape(x, (48, 48, 1)) for x in X_val])
X_test = np.array([np.reshape(x, (48, 48, 1)) for x in X_test])

# Perform one-hot encoding for emotion labels
num_classes = 7  # 7 classes for 7 emotions
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

# Now you have X_train, y_train, X_val, y_val, X_test, and y_test ready for training and evaluation.


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define your emotion recognition model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(7, activation='softmax')  # 7 classes for 7 emotions
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 4, 4, 128)         0

In [6]:
# Train the emotion recognition model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
718/718 [==============================] - 47s 65ms/step - loss: 1.6992 - accuracy: 0.3141 - val_loss: 1.5043 - val_accuracy: 0.4286
Epoch 2/10
718/718 [==============================] - 47s 66ms/step - loss: 1.4153 - accuracy: 0.4555 - val_loss: 1.3577 - val_accuracy: 0.4789
Epoch 3/10
718/718 [==============================] - 47s 65ms/step - loss: 1.2844 - accuracy: 0.5127 - val_loss: 1.2493 - val_accuracy: 0.5279
Epoch 4/10
718/718 [==============================] - 46s 65ms/step - loss: 1.1962 - accuracy: 0.5493 - val_loss: 1.2599 - val_accuracy: 0.5280
Epoch 5/10
718/718 [==============================] - 46s 65ms/step - loss: 1.1210 - accuracy: 0.5750 - val_loss: 1.2066 - val_accuracy: 0.5465
Epoch 6/10
718/718 [==============================] - 47s 65ms/step - loss: 1.0532 - accuracy: 0.6013 - val_loss: 1.1948 - val_accuracy: 0.5535
Epoch 7/10
718/718 [==============================] - 46s 64ms/step - loss: 0.9787 - accuracy: 0.6358 - val_loss: 1.2159 - val_accuracy:

In [7]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

225/225 [==============================] - 4s 19ms/step - loss: 1.3495 - accuracy: 0.5489
Test Loss: 1.3494791984558105
Test Accuracy: 0.5488994121551514


In [8]:
# Save the model
model.save('/content/drive/MyDrive/models/emotion_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
